# Deep Knowledge Tracing (DKT) 구현

## 소개
Deep Knowledge Tracing(DKT)은 교육 분야에서 학생들의 지식 상태를 추적하는 딥러닝 기반 방법론입니다. 전통적인 Knowledge Tracing과 달리, DKT는 순환 신경망(RNN)을 사용하여 학생의 학습 과정을 모델링합니다. 이를 통해 학생이 특정 문제를 해결할 수 있는 확률을 예측할 수 있습니다.

## 환경 설정
AWS Inferentia2(INF2)는 딥러닝 추론을 위해 특별히 설계된 가속기입니다. 이를 활용하기 위해 필요한 라이브러리들을 설치합니다.

In [1]:
# Neuron SDK 및 PyTorch NeuronX 설치
!pip install torch-neuronx torchvision neuronx-cc[tensorflow] -U

# 기타 필요한 패키지 설치
!pip install numpy pandas scikit-learn matplotlib -U

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached https://pip.repos.neuron.amazonaws.com/torch-neuronx/torch_neuronx-2.1.2.2.3.2-py3-none-any.whl (2.6 MB)
  Using cached torchvision-0.19.1-cp38-cp38-manylinux1_x86_64.whl.metadata (6.0 kB)
  Using cached torch-2.1.2-cp38-cp38-manylinux1_x86_64.whl.metadata (25 kB)
INFO: pip is looking at multiple versions of torch-neuronx to determine which version is compatible with other requirements. This could take a while.
  Using cached https://pip.repos.neuron.amazonaws.com/torch-neuronx/torch_neuronx-2.1.2.2.3.1-py3-none-any.whl (2.6 MB)
  Using cached https://pip.repos.neuron.amazonaws.com/torch-neuronx/torch_neuronx-2.1.2.2.3.0-py3-none-any.whl (2.6 MB)
  Using cached https://pip.repos.neuron.amazonaws.com/torch-neuronx/torch_neuronx-2.1.2.2.2.0-py3-none-any.whl (2.5 MB)
  Using cached https://pip.repos.neuron.amazonaws.com/torch-neuronx/torch_neuronx-2.1.2.2.1.0-py3-none-any.whl (2.5 MB)
INFO: 

## 라이브러리 임포트

필요한 라이브러리를 임포트합니다.
- numpy: 수치 연산을 위한 기본 라이브러리
- pandas: 데이터 처리 및 분석
- torch: 딥러닝 모델 구현
- torch_neuronx: AWS Inferentia2 최적화
- sklearn: 성능 평가 및 데이터 전처리

In [28]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch_neuronx
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

## 데이터셋 소개
ASSISTments 2009 데이터셋은 교육 데이터 마이닝 분야에서 널리 사용되는 공개 데이터셋입니다. 이 데이터셋은 다음과 같은 특징을 가집니다:
- 학생들의 수학 문제 풀이 기록
- 문제 ID, 학생 ID, 정답 여부 등의 정보 포함
- 실제 교육 환경에서 수집된 데이터

In [4]:
# 데이터셋 경로 설정 (실제 경로로 변경하세요)
data_path = 'datasets/ASSIST2009/skill_builder_data.csv'

# 데이터 로드
data = pd.read_csv(data_path, encoding='ISO-8859-1', low_memory=False)
data.head()

,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537,277618,64525,33139,51424,1,1,1,32454,tutor,...,0,3,32454,30799,NaN,26,0,NaN,1,1.0
1,33022709,277618,64525,33150,51435,1,1,1,4922,tutor,...,0,3,4922,30799,NaN,55,0,NaN,2,2.0
2,35450204,220674,70363,33159,51444,1,0,2,25390,tutor,...,0,3,42000,30799,NaN,88,0,NaN,1,1.0
3,35450295,220674,70363,33110,51395,1,1,1,4859,tutor,...,0,3,4859,30059,NaN,41,0,NaN,2,2.0
4,35450311,220674,70363,33196,51481,1,0,14,19813,tutor,...,3,4,124564,30060,NaN,65,0,0.0,3,3.0


### 데이터 전처리

모델 학습에 필요한 형태로 데이터를 전처리합니다.

In [5]:
# 필요한 컬럼 선택
data = data[['user_id', 'problem_id', 'correct']]

# 결측치 제거
data = data.dropna()

# ID 인코딩
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
data['user_id'] = user_encoder.fit_transform(data['user_id'])

item_encoder = LabelEncoder()
data['problem_id'] = item_encoder.fit_transform(data['problem_id'])

# 문제 수와 학생 수 확인
num_students = data['user_id'].nunique()
num_questions = data['problem_id'].nunique()

print(f'학생 수: {num_students}')
print(f'문제 수: {num_questions}')

학생 수: 4217
문제 수: 26688


### 시퀀스 데이터 생성

학생별로 문제 풀이 이력을 시퀀스로 만듭니다.

In [6]:
# 학생별로 그룹화
grouped = data.groupby('user_id')

# 시퀀스 생성
sequences = []

for _, group in grouped:
    seq = list(zip(group['problem_id'].values, group['correct'].values))
    sequences.append(seq)

print(f'총 시퀀스 수: {len(sequences)}')

총 시퀀스 수: 4217


### Dataset 및 DataLoader 구성

모델 학습을 위한 PyTorch Dataset과 DataLoader를 정의합니다.

In [7]:
class DKTDataset(Dataset):
    def __init__(self, sequences, num_questions, seq_len=100):
        self.sequences = sequences
        self.num_questions = num_questions
        self.seq_len = seq_len
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = self.sequences[idx]
        
        seq = seq[-self.seq_len:]
        
        x = np.zeros(self.seq_len, dtype=int)
        y = np.zeros(self.seq_len, dtype=int)
        
        for i, (q, r) in enumerate(seq):
            x[i] = q + self.num_questions * r
            y[i] = q
        
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

# 데이터셋 생성
dataset = DKTDataset(sequences, num_questions)

# 훈련셋과 테스트셋 분할
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 생성
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=True)

## DKT 모델 아키텍처
DKT 모델은 다음과 같은 구조로 이루어져 있습니다:
1. 임베딩 층: 문제 ID를 벡터로 변환
2. LSTM 층: 시퀀스 데이터 처리
3. 출력 층: 다음 문제 정답 확률 예측

DKT 모델은 LSTM(Long Short-Term Memory) 신경망을 기반으로 하며, 학생의 과거 문제 풀이 이력을 입력으로 받아 다음 문제의 정답 확률을 예측합니다. LSTM을 기반으로 한 DKT 모델을 정의합니다.

In [8]:
class DKTModel(nn.Module):
    def __init__(self, num_questions, hidden_size):
        super(DKTModel, self).__init__()
        self.num_questions = num_questions
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(num_questions * 2, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_questions)
    
    def forward(self, x):
        embed = self.embedding(x)
        h, _ = self.lstm(embed)
        out = self.fc(h)
        preds = torch.sigmoid(out)
        return preds

## 모델 학습

모델을 학습시키는 코드를 작성합니다. 학습 과정에서 고려할 사항들:
- 배치 크기 선택
- 학습률 조정
- 손실 함수 (BCE Loss)
- 옵티마이저 (Adam)
- 에포크 수

In [9]:
# 디바이스 설정 (INF2에서는 CPU로 설정)
device = torch.device('cpu')

# 하이퍼파라미터 설정
num_epochs = 10
hidden_size = 100

# 모델 초기화
model = DKTModel(num_questions, hidden_size).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
train_losses = []

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        preds = model(x)
        
        # 타깃 생성
        target_mask = (y != 0)
        target_indices = y[target_mask]
        
        preds = preds[target_mask, target_indices]
        targets = (x[target_mask] >= num_questions).float()
        
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# 학습된 모델 저장
torch.save(model.state_dict(), 'dkt_model.pth')

Epoch [1/10], Loss: 0.6603
Epoch [2/10], Loss: 0.5177
Epoch [3/10], Loss: 0.2917
Epoch [4/10], Loss: 0.1760
Epoch [5/10], Loss: 0.1177
Epoch [6/10], Loss: 0.0840
Epoch [7/10], Loss: 0.0631
Epoch [8/10], Loss: 0.0489
Epoch [9/10], Loss: 0.0390
Epoch [10/10], Loss: 0.0319


## 모델 컴파일 (Neuron 최적화)

학습된 모델을 INF2에서 추론할 수 있도록 `torch_neuronx`를 사용하여 컴파일합니다.

예시 입력(example_inputs)을 생성하는 이유는 다음과 같습니다:

1. **모델 컴파일을 위한 입력 형태 정의**
   - torch_neuronx.trace() 함수는 모델을 AWS Inferentia2에 최적화된 형태로 컴파일하기 위해 입력 텐서의 구체적인 형태(shape)와 데이터 타입을 알아야 합니다.
   - 예시 입력을 통해 모델이 실제로 받게 될 데이터의 구조를 미리 알려주는 것입니다.

2. **컴파일 최적화**
   ```python
   example_inputs = torch.randint(0, num_questions * 2, (batch_size, 100), dtype=torch.long)
   ```
   - batch_size: 한 번에 처리할 데이터 수 (여기서는 64)
   - 100: 시퀀스 길이
   - num_questions * 2: 가능한 입력값의 범위 (문제 ID와 정답 여부 조합)

3. **실행 그래프 생성**
   - 컴파일러는 이 예시 입력을 사용하여 모델의 연산 그래프를 최적화합니다.
   - 실제 추론 시에 사용될 연산 패턴을 미리 분석하고 최적화합니다.

4. **하드웨어 최적화**
   ```python
   model_neuron = torch_neuronx.trace(model, example_inputs)
   ```
   - AWS Inferentia2 하드웨어에 맞게 연산을 최적화합니다.
   - 메모리 사용량과 연산 속도를 개선합니다.

예시:
```python
# 실제 배치 크기가 64이고, 시퀀스 길이가 100인 경우
batch_size = 64
seq_length = 100
num_questions = 100  # 전체 문제 수

# 예시 입력 생성
example_inputs = torch.randint(
    low=0,  # 최소값
    high=num_questions * 2,  # 최대값 (문제 수 * 2)
    size=(batch_size, seq_length),  # 입력 텐서의 크기
    dtype=torch.long  # 데이터 타입
)

# 이 예시 입력으로 모델 컴파일
model_neuron = torch_neuronx.trace(model, example_inputs)
```

이렇게 생성된 예시 입력은 실제 데이터가 아닌 더미 데이터이지만, 모델 컴파일과 최적화에 필요한 모든 구조적 정보를 포함하고 있습니다.

In [10]:
# 모델 로드
model.load_state_dict(torch.load('dkt_model.pth'))
model.eval()

# 예시 입력 생성
example_inputs = torch.randint(0, num_questions * 2, (batch_size, 100), dtype=torch.long)

# Neuron으로 컴파일
model_neuron = torch_neuronx.trace(model, example_inputs)

# 컴파일된 모델 저장
model_neuron.save('dkt_model_neuron.pt')

2024-11-25 14:22:00.000511:  18793  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-11-25 14:22:00.000512:  18793  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: ['neuronx-cc', '--target=trn1', 'compile', '--framework', 'XLA', '/tmp/ec2-user/neuroncc_compile_workdir/f1751cb8-0a67-4cb8-bc6a-c417a2e51eee/model.MODULE_10466109672838001554+d41d8cd9.hlo.pb', '--output', '/tmp/ec2-user/neuroncc_compile_workdir/f1751cb8-0a67-4cb8-bc6a-c417a2e51eee/model.MODULE_10466109672838001554+d41d8cd9.neff', '--verbose=35']
.
Compiler status PASS
2024-11-25 14:22:02.000881:  18863  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-11-25 14:22:02.000882:  18863  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: ['neuronx-cc', '--target=trn1', 'compile', '--framework', 'XLA', '/tmp/ec2-user/neuroncc_compile_workdir/20158125-f6fd-4581-b4a0-c79305a82d2d/model.MODULE_10076059576887794638+d41d8cd9.hlo.pb', '--output', '/tmp/ec2-user/neuroncc_comp

## 성능 평가

컴파일된 모델을 사용하여 INF2에서 추론을 수행합니다. DKT 모델의 성능은 다음 지표로 평가됩니다:
- AUC (Area Under the ROC Curve)
- 정확도 (Accuracy)
- 손실 함수 값의 변화

AWS Inferentia 하드웨어에서 모델을 실행할 때는 입력 텐서의 크기가 컴파일 시 지정된 크기와 정확히 일치해야 합니다.

In [29]:
# 컴파일된 모델 로드
model_neuron = torch.jit.load('dkt_model_neuron.pt')
model_neuron.eval()

all_preds = []
all_targets = []
total_samples = 0
total_sequences = 0

with torch.no_grad():
    for batch_idx, (x, y) in enumerate(test_loader):
        x = x.to(device)
        y = y.to(device)
        preds = model_neuron(x)
        
        batch_samples = 0
        for i in range(len(y)):
            target_mask = (y[i] != 0)
            target_indices = y[i][target_mask]
            pred = preds[i][target_mask, target_indices]
            
            # 현재 시퀀스의 실제 문제 수 계산
            num_problems = len(target_indices)
            batch_samples += num_problems
            
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend((x[i][target_mask] >= num_questions).cpu().numpy())
        
        total_samples += batch_samples
        total_sequences += len(y)

print(f'\n[테스트 통계]')
print(f'총 시퀀스 수: {total_sequences}')
print(f'총 문제 수: {total_samples}')
print(f'시퀀스당 평균 문제 수: {total_samples/total_sequences:.2f}')
print(f'전체 데이터 크기: {len(all_preds)}')

# 성능 평가
auc = roc_auc_score(all_targets, all_preds)
print(f'\n테스트 성능:')
print(f'AUC: {auc * 100:.2f}%')


[테스트 통계]
총 시퀀스 수: 832
총 문제 수: 35537
시퀀스당 평균 문제 수: 42.71
전체 데이터 크기: 35537

테스트 성능:
AUC: 99.17%


## 결론

이 노트북에서는 DKT 모델을 PyTorch와 AWS Inferentia2를 사용하여 구현하고, `torch-neuronx`를 통해 INF2에서 추론을 수행했습니다. 초보자도 이해할 수 있도록 단계별로 설명하였으며, INF2의 성능을 활용하여 효율적인 추론을 달성할 수 있습니다.

## 참고 자료

- [Deep Knowledge Tracing 논문](https://papers.nips.cc/paper/2015/file/efc3f4b5768f887a677ce7f1dba75504-Paper.pdf)
- [ASSISTments 데이터셋](https://sites.google.com/site/assistmentsdata/home/assistment-2009-2010-data)
- [AWS Neuron SDK 문서](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/neuron-guide/neuron-frameworks/pytorch-neuronx/tutorials/torch-neuronx.html)